In [0]:
import os
os.environ['KAGGLE_USERNAME'] = ""
os.environ['KAGGLE_KEY'] = ""
!kaggle competitions download -c competitive-data-science-final-project

In [0]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split

items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')
test = pd.read_csv('test.csv.gz')
train = pd.read_csv('sales_train.csv.gz')
sample_submission = pd.read_csv('sample_submission.csv.gz')

# convert train dataset string date to date time
train.date = pd.to_datetime(train.date,format='%d.%m.%Y')

In [0]:
# joined data frame train

train_joined = (
    train
    .join(shops.set_index('shop_id'),on='shop_id')
    .join(items.set_index('item_id'),on='item_id')
    .join(item_categories.set_index('item_category_id'),on='item_category_id')  
)

In [0]:
sale_month = train_joined[['shop_id','item_id','date_block_num','item_cnt_day']].groupby(['shop_id','item_id','date_block_num']).sum()

In [0]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

_temp_train = train_joined.drop(['item_cnt_day','date'],axis=1)
y = train_joined.item_cnt_day

X = _temp_train.copy()

X.fillna(-999)

X,y = X[::40],y[::40]

for c in _temp_train.columns[_temp_train.dtypes == 'object']:
    X[c] = X[c].factorize()[0]
    
rf = RandomForestClassifier()
rf.fit(X,y)

In [0]:
import matplotlib.pyplot as plt
plt.plot(rf.feature_importances_)
plt.xticks(np.arange(X.shape[1]), X.columns.tolist(), rotation=90);

In [0]:
train_joined.head(5)
train_joined['dayofweek'] = train_joined.date.dt.dayofweek

In [96]:
# duplicated
train_joined.duplicated().sum()

6

In [0]:
#train_joined[['dayofweek','item_cnt_day','date_block_num']].groupby(['dayofweek','date_block_num']).sum().plot()
#train_joined[['dayofweek','item_cnt_day']].groupby(['dayofweek']).sum().plot()
#train_joined[['date_block_num','item_cnt_day']].groupby(['date_block_num']).sum().plot()
#train_joined[['item_id','item_cnt_day']].groupby(['item_id']).sum().plot()
#train_joined[['item_id','item_cnt_day']].groupby(['item_id']).sum().idxmax()
#train_joined[train_joined.item_id==20949].nunique()
_shps = set(train_joined.item_category_name)
_shps
#20949,71